In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz 
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

findspark.init()
findspark.find()

'/content/spark-3.1.2-bin-hadoop2.7'

# Logistic Regression

- 참고 : https://spark.apache.org/docs/latest/ml-classification-regression.html

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

In [5]:
from pyspark.ml.classification import LogisticRegression

## Load training data

In [7]:
!ls /content/spark-3.1.2-bin-hadoop2.7/sample_libsvm_data.txt

ls: cannot access '/content/spark-3.1.2-bin-hadoop2.7/sample_libsvm_data.txt': No such file or directory


In [13]:
my_data = spark.read.format("libsvm").load("/content/spark-3.1.2-bin-hadoop2.7/data/mllib/sample_libsvm_data.txt")

In [14]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



## Fit the model

In [16]:
model = LogisticRegression()

In [18]:
fitted_model = model.fit(my_data)

In [20]:
log_summary = fitted_model.summary

In [21]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [22]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111048...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126974...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170280...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200599...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695573...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102630...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

## train_data, test_data 데이터셋 분리

In [23]:
train, test = my_data.randomSplit([0.7, 0.3])

## Fit the model

In [24]:
final_model = LogisticRegression()

In [25]:
fit_final = final_model.fit(train)

## Evaluation

In [26]:
prediction_and_labels = fit_final.evaluate(train)

In [28]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[23.4730613312538...|[0.99999999993605...|       0.0|
|  0.0|(692,[98,99,100,1...|[27.5581854406991...|[0.99999999999892...|       0.0|
|  0.0|(692,[100,101,102...|[19.2602546077710...|[0.99999999568105...|       0.0|
|  0.0|(692,[121,122,123...|[22.9836776122599...|[0.99999999989569...|       0.0|
|  0.0|(692,[122,123,124...|[19.7150728796769...|[0.99999999725935...|       0.0|
|  0.0|(692,[122,123,148...|[20.7479806713662...|[0.99999999902441...|       0.0|
|  0.0|(692,[123,124,125...|[32.3557626994730...|[0.99999999999999...|       0.0|
|  0.0|(692,[123,124,125...|[21.3015946779741...|[0.99999999943916...|       0.0|
|  0.0|(692,[124,125,126...|[22.8133657164778...|[0.99999999987632...|       0.0|
|  0.0|(692,[124

In [33]:
from pyspark.ml.evaluation import ( BinaryClassificationEvaluator, 
                                 MulticlassClassificationEvaluator)

In [34]:
my_eval = BinaryClassificationEvaluator()

In [35]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [36]:
my_final_roc

1.0